In [1]:
import typing

import rich
from rich.progress import track

import pandas
import transformers
import evaluate

import cltrier_lib

In [2]:
pandas.set_option('display.max_colwidth', None)

In [3]:
RAW_DATASET: str = "../data/interim/twitter.german.dataset.enriched.csv"
TEST_SIZE: int = 50

MODELS: typing.Dict[str, str] = {
    "base": "meta-llama/Llama-3.2-3B-Instruct", 
    "adapter": "simon-muenker/Llama-3.2-3B-Instruct-OSN-posts"
}

In [4]:
pipelines: typing.Dict[str, transformers.Pipeline] = {
    label: transformers.pipeline("text-generation", slug, device=f"cuda:{1}")
    for n, (label, slug) in enumerate(MODELS.items())
}

pipelines["adapter"].model.load_adapter(MODELS["adapter"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:1


In [5]:
dataset: typing.List[cltrier_lib.inference.schemas.Chat] = [
    cltrier_lib.inference.schemas.Chat(messages=[
        cltrier_lib.inference.schemas.Message(role="system", content=f"You are a social media user with a political {row['leaning_post']} leaning. Post a Tweet about the following topic:"),
        cltrier_lib.inference.schemas.Message(role="user", content=row["topics_post"]),
        cltrier_lib.inference.schemas.Message(role="assistant", content=row["text_post"])
    ])
    for _, row in pandas.read_csv(RAW_DATASET, index_col=0).iterrows()
]
dataset[:3]

[Chat(messages=[Message(role='system', content='You are a social media user with a political neutral leaning. Post a Tweet about the following topic:'), Message(role='user', content='Klima, eFuels, Verbrenner'), Message(role='assistant', content='Nicht der #Verbrenner schadet dem #Klima, sondern der fossile Sprit, mit dem er fährt. Wir haben diese Woche den Weg für klimaneutrale #eFuels freigemacht. Damit könnten die mehr als 45 Mio. Diesel- und Benzin-Fahrzeuge auf unseren Straßen in Zukunft klimaneutral unterwegs sein.')]),
 Chat(messages=[Message(role='system', content='You are a social media user with a political right leaning. Post a Tweet about the following topic:'), Message(role='user', content='Kernkraft, Altparteien, AfD'), Message(role='assistant', content='Wo waren die ganzen plötzlichen #Kernkraftbefürworter in #Altparteien, Verbänden &amp; Medien in den letzten Jahren? Warum stimmte die #umfaller: #fdp bis zuletzt im Bundestag gegen Laufzeitverlängerungen? Fakt ist: nur d

In [6]:
responses = []

for idx, chat in enumerate(track(dataset[:TEST_SIZE])):
    responses.append(
        dict(
            id=idx,
            model="human",
            reply=chat[-1].content
        )
    )

    for model, pipeline in pipelines.items():
        reply = pipeline(
            pipeline.tokenizer.apply_chat_template(chat[:-1], tokenize=False), 
            max_new_tokens=128,
            return_full_text=False
        )[0]["generated_text"].split("\n\n")[1]

        responses.append(
            dict(
                id=idx,
                model=model,
                reply=reply
            )
        )

Output()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [29]:
predictions: pandas.DataFrame = (
    pandas.DataFrame(responses)
    .set_index("id")
    .pivot(columns=["model"])
)
predictions.head().style.set_properties(**{'text-align': 'left'})

In [ ]:
scores_base = evaluate.load("bleu").compute(
    references=predictions[("reply", "human")].tolist(),
    predictions=predictions[("reply", "base")].tolist(),
    smooth=True
)
rich.print(scores_base)

{
    'bleu': 0.007841075059461276,
    'precisions': [0.15433320142461418, 0.014129995962858296, 0.0020601565718994645, 0.0008413967185527977],
    'brevity_penalty': 1.0,
    'length_ratio': 1.2731854838709677,
    'translation_length': 2526,
    'reference_length': 1984
}

In [11]:
scores_adapter = evaluate.load("bleu").compute(
    references=predictions[("reply", "human")].tolist(),
    predictions=predictions[("reply", "adapter")].tolist(),
    smooth=True
)
rich.print(scores_adapter)

{
    'bleu': 0.024389163676571863,
    'precisions': [0.2159090909090909, 0.036744186046511626, 0.011428571428571429, 0.003902439024390244],
    'brevity_penalty': 1.0,
    'length_ratio': 1.108366935483871,
    'translation_length': 2199,
    'reference_length': 1984
}

In [32]:
print(
    pandas.DataFrame({
        "prompt": scores_base,
        "fine-tuned": scores_adapter
    }).to_latex()
)

\begin{tabular}{lll}
\toprule
 & prompt & fine-tuned \\
\midrule
bleu & 0.007841 & 0.024389 \\
precisions & [0.15433320142461418, 0.014129995962858296, 0.0020601565718994645, 0.0008413967185527977] & [0.2159090909090909, 0.036744186046511626, 0.011428571428571429, 0.003902439024390244] \\
brevity_penalty & 1.000000 & 1.000000 \\
length_ratio & 1.273185 & 1.108367 \\
translation_length & 2526 & 2199 \\
reference_length & 1984 & 1984 \\
\bottomrule
\end{tabular}

